In [1]:
import json
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
#from PIL import Image as pil_image
#from PIL.ImageDraw import Draw

In [2]:
def create_bb(label_df, img):

    for column in label_df.columns:
        x = label_df[column]['box'][0]
        y = label_df[column]['box'][1]
        w = label_df[column]['box'][2] - label_df[column]['box'][0]
        h = label_df[column]['box'][3] - label_df[column]['box'][1]

        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(img,str(label_df[column]['label']),(x+w+10,y+h),0,0.3,(0,255,0))

In [3]:
files = os.listdir('./Flight_5_with_ground_truth/')
images = [x for x in files if '.jpg' in x]
label_data = [x for x in files if '.json' in x]

In [4]:
for x in range(0, len(images)):
    # label_data contains a list of labeling data filenames
    with open('./Flight_5_with_ground_truth/'+label_data[x]) as json_data:
        boxes = pd.DataFrame(json.load(json_data))

    # images contains a list of image filenames
    cv_img = cv2.imread('./Flight_5_with_ground_truth/'+images[x])
    
    #creates bounding boxes
    create_bb(boxes, cv_img)
    
    if x == 1 or x == 699:
        cv2.imshow('image',cv_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
#with open('./Flight_5_with_ground_truth/frame_000000_GT.json') as json_data:

In [5]:
import random
import numpy as np
from scipy.ndimage import affine_transform
from keras.preprocessing.image import img_to_array
from PIL import Image as pil_image
from PIL.ImageDraw import Draw
from os.path import isfile

def read_raw_image(p):
    return pil_image.open((p))

# Read an image as black&white numpy array
def read_array(p):
    img = read_raw_image(p).convert('L')
    return img_to_array(img)

test = read_array('./Flight_5_with_ground_truth/frame_000000.tif')
test2 = cv2.imread('./Flight_5_with_ground_truth/frame_000000.tif')
test3 = np.array(pil_image.open('./Flight_5_with_ground_truth/frame_000000.tif'))
#    print('True')
print(test.shape)
print(test2.shape)
print(test3.shape)
print(np.ravel(test).shape)
print(np.ravel(test3))
print(np.ravel(test2))


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(1024, 1280, 1)
(1024, 1280, 3)
(1024, 1280)
(1310720,)
[104  98  97 ... 230 226 223]
[104 104 104 ... 223 223 223]


In [6]:
img_shape  = (1024,1280,3)
anisotropy = 2.15

In [7]:
import random
import numpy as np
from scipy.ndimage import affine_transform
from keras.preprocessing.image import img_to_array
from keras import backend as K

# Read an image as black&white numpy array
def read_array(p):
    img = read_raw_image(p).convert('L')
    return img_to_array(img)

def build_transform(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    rotation        = np.deg2rad(rotation)
    shear           = np.deg2rad(shear)
    rotation_matrix = np.array([[np.cos(rotation), np.sin(rotation), 0], [-np.sin(rotation), np.cos(rotation), 0], [0, 0, 1]])
    shift_matrix    = np.array([[1, 0, height_shift], [0, 1, width_shift], [0, 0, 1]])
    shear_matrix    = np.array([[1, np.sin(shear), 0], [0, np.cos(shear), 0], [0, 0, 1]])
    zoom_matrix     = np.array([[1.0/height_zoom, 0, 0], [0, 1.0/width_zoom, 0], [0, 0, 1]])
    shift_matrix    = np.array([[1, 0, -height_shift], [0, 1, -width_shift], [0, 0, 1]])
    return np.dot(np.dot(rotation_matrix, shear_matrix), np.dot(zoom_matrix, shift_matrix))

# Compute the coordinate transformation required to center the pictures, padding as required.
def center_transform(affine, input_shape):
    hi, wi = float(input_shape[0]), float(input_shape[1])
    ho, wo = float(img_shape[0]), float(img_shape[1])
    top, left, bottom, right = 0, 0, hi, wi
    if wi/hi/anisotropy < wo/ho: # input image too narrow, extend width
        w     = hi*wo/ho*anisotropy
        left  = (wi-w)/2
        right = left + w
    else: # input image too wide, extend height
        h      = wi*ho/wo/anisotropy
        top    = (hi-h)/2
        bottom = top + h
    center_matrix   = np.array([[1, 0, -ho/2], [0, 1, -wo/2], [0, 0, 1]])
    scale_matrix    = np.array([[(bottom - top)/ho, 0, 0], [0, (right - left)/wo, 0], [0, 0, 1]])
    decenter_matrix = np.array([[1, 0, hi/2], [0, 1, wi/2], [0, 0, 1]])
    return np.dot(np.dot(decenter_matrix, scale_matrix), np.dot(affine, center_matrix))

# Apply an affine transformation to an image represented as a numpy array.
def transform_img(x, affine):
    matrix   = affine[:2,:2]
    offset   = affine[:2,2]
    x        = np.moveaxis(x, -1, 0)
    channels = [affine_transform(channel, matrix, offset, output_shape=img_shape[:-1], order=1,
                                 mode='constant', cval=np.average(channel)) for channel in x]
    return np.moveaxis(np.stack(channels, axis=0), 0, -1)

# Read an image for validation, i.e. without data augmentation.
def read_for_validation(p):
    x  = read_array(p)
    t  = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    t  = center_transform(t, x.shape)
    x  = transform_img(x, t)
    x -= np.mean(x, keepdims=True)
    x /= np.std(x, keepdims=True) + K.epsilon()
    return x,t 

# Read an image for training, i.e. including a random affine transformation
def read_for_training(p):
    # changed to use opencv instead of PIL
    # x  = read_array(p)
    x = cv2.imread(p)
    t  = build_transform(
            random.uniform(-5, 5),
            random.uniform(-5, 5),
            random.uniform(0.9, 1.0),
            random.uniform(0.9, 1.0),
            random.uniform(-0.05*img_shape[0], 0.05*img_shape[0]),
            random.uniform(-0.05*img_shape[1], 0.05*img_shape[1]))
    t  = center_transform(t, x.shape)
    x  = transform_img(x, t)
    x = x - np.mean(x, keepdims=True)
    x /= np.std(x, keepdims=True) + K.epsilon()
    return x,t   

# Transform corrdinates according to the provided affine transformation
def coord_transform(list, trans):
    result = []
    for x,y in list:
        y,x,_ = trans.dot([y,x,1]).astype(np.int)
        result.append((x,y))
    return result

In [8]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data, test_size=200, random_state=1)
train += train
train += train
train += train
train += train
len(train),len(val)

NameError: name 'data' is not defined

In [9]:
read_for_training('./Flight_5_with_ground_truth/frame_000000.tif')

(array([[[0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597],
         ...,
         [0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597]],
 
        [[0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597],
         ...,
         [0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597]],
 
        [[0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597],
         ...,
         [0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040597]],
 
        ...,
 
        [[0.00040597, 0.00040597, 0.00040597],
         [0.00040597, 0.00040597, 0.00040

In [106]:
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
from keras import backend as K
from keras.preprocessing.image import array_to_img
from numpy.linalg import inv as mat_inv

def show_whale(imgs, per_row=5):
    n         = len(imgs)
    rows      = (n + per_row - 1)//per_row
    cols      = min(per_row, n)
    fig, axes = plt.subplots(rows,cols, figsize=(24//per_row*cols,24//per_row*rows))
    for ax in axes.flatten(): ax.axis('off')
    for i,(img,ax) in enumerate(zip(imgs, axes.flatten())): ax.imshow(img.convert('RGB'))

val_a = np.zeros((len(val),)+img_shape,dtype=K.floatx()) # Preprocess validation images 
val_b = np.zeros((len(val),4),dtype=K.floatx()) # Preprocess bounding boxes
for i,(p,coords) in enumerate(tqdm_notebook(val)):
    img,trans      = read_for_validation(p)
    coords         = coord_transform(coords, mat_inv(trans))
    x0,y0,x1,y1    = bounding_rectangle(coords)
    val_a[i,:,:,:] = img
    val_b[i,0]     = x0
    val_b[i,1]     = y0
    val_b[i,2]     = x1
    val_b[i,3]     = y1

idx  = 1
img  = array_to_img(val_a[idx])
img  = img.convert('RGB')
draw = Draw(img)
draw.rectangle(val_b[idx], outline='red')
show_whale([read_raw_image(val[idx][0]), img], per_row=2)

NameError: name 'val' is not defined

In [ ]:
from keras.utils import Sequence

class TrainingData(Sequence):
    def __init__(self, batch_size=32):
        super(TrainingData, self).__init__()
        self.batch_size = batch_size
    def __getitem__(self, index):
        start = self.batch_size*index;
        end   = min(len(train), start + self.batch_size)
        size  = end - start
        a     = np.zeros((size,) + img_shape, dtype=K.floatx())
        b     = np.zeros((size,4), dtype=K.floatx())
        for i,(p,coords) in enumerate(train[start:end]):
            img,trans   = read_for_training(p)
            coords      = coord_transform(coords, mat_inv(trans))
            x0,y0,x1,y1 = bounding_rectangle(coords)
            a[i,:,:,:]  = img
            b[i,0]      = x0
            b[i,1]      = y0
            b[i,2]      = x1
            b[i,3]      = y1
        return a,b
    def __len__(self):
        return (len(train) + self.batch_size - 1)//self.batch_size

random.seed(1)
a, b = TrainingData(batch_size=5)[1]
img  = array_to_img(a[0])
img  = img.convert('RGB')
draw = Draw(img)
draw.rectangle(b[0], outline='red')
show_whale([read_raw_image(train[0][0]), img], per_row=2)

In [93]:
def expand_path(p):
    if isfile('./train/' + p): return './train/' + p
    if isfile('./test/' + p): return './test/' + p
    return p

def read_raw_image(p):
    return pil_image.open(expand_path(p))

def draw_dot(draw, x, y):
    draw.ellipse(((x-5,y-5),(x+5,y+5)), fill='red', outline='red')

def draw_dots(draw, coordinates):
    for x,y in coordinates: draw_dot(draw, x, y)

def bounding_rectangle(list):
    x0, y0 = list[0]
    x1, y1 = x0, y0
    for x,y in list[1:]:
        x0 = min(x0, x)
        y0 = min(y0, y)
        x1 = max(x1, x)
        y1 = max(y1, y)
    return x0,y0,x1,y1

In [18]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()